## Servicio Social – Comparativa de Modelos de Visión Computacional para Detección de Contaminantes en Cosechas de Chile 🌶️

In [42]:
import torch

print(torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("CUDA:", torch.cuda.is_available())

2.9.0+cu126
CUDA: True


In [43]:
# montando drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
import os
# carpetas de las imagenes
BASE_PATH = "/content/drive/MyDrive/ServicioSocial"
DATA_YAML = os.path.join(BASE_PATH, "data.yaml")

## Integridad de los datos

In [45]:
import os
data_types = ["train", "valid", "test"]

for split in data_types:
    images_path = os.path.join(BASE_PATH, split, "images")
    labels_path = os.path.join(BASE_PATH, split, "labels")

    num_images = len([
        f for f in os.listdir(images_path)
        if f.lower().endswith((".png", ".jpg", ".jpeg"))
    ])

    num_labels = len(os.listdir(labels_path))

    print(f"{split.upper()}:")
    print(f"  Imágenes: {num_images}")
    print(f"  Labels:   {num_labels}")


TRAIN:
  Imágenes: 3704
  Labels:   3704
VALID:
  Imágenes: 1236
  Labels:   1236
TEST:
  Imágenes: 1232
  Labels:   1232


### Implementacion del modelo: YoloV8

In [ ]:

from ultralytics import YOLO
import torch
import os

# instancia
model = YOLO("yolov8n.pt")  # pesos preentrenados

# hyperparametros del entrenamientos
model.train(
    data=DATA_YAML,
    epochs=70,
    imgsz=640,
    batch=16,
    device=0  # usa GPU automáticamente
)





New https://pypi.org/project/ultralytics/8.4.2 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.4.1 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/ServicioSocial/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=70, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train5, nbs=64,

In [ ]:
# VALIDACIÓN
metrics = model.val(data=DATA_YAML, split="test")
print("============= Metricas =================")
print("mAP@0.5:", metrics.box.map50)
print("mAP@0.5:0.95:", metrics.box.map)
print("Mean Precision:", metrics.box.mp)
print("Mean Recall:", metrics.box.mr)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_box_loss(results_csv_path):
    df = pd.read_csv(results_csv_path)

    plt.figure()
    plt.plot(df["epoch"], df["train/box_loss"], label="Train Box Loss")
    plt.plot(df["epoch"], df["val/box_loss"], label="Val Box Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Box Loss")
    plt.title("Box Loss durante el entrenamiento")
    plt.legend()
    plt.grid(True)
    plt.show()


plot_box_loss("runs/detect/train/results.csv")

In [ ]:
# CARGAR MEJOR MODELO

best_model_path = "runs/detect/train/weights/best.pt"
model = YOLO(best_model_path)

# INFERENCIA
results = model.predict(
    source=f"{BASE_PATH}/test/images",
    conf=0.25,
    save=True
)

print("Detección finalizada")